In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle


## load the dataset
data= pd.read_csv("Churn_Modelling.csv")
data.head()

## Preprocess the data
### Drop irrelevant column
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data

## encode categorical variables
'''
👉 fit()

Reads unique values and assigns a number:

"Female" → 0

"Male" → 1
(Alphabetical order: Female comes before Male)

👉 transform()

Replaces text with numbers in the column.
'''
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

## Onehot encode 'geogrphy column'
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder_geo = OneHotEncoder()
geo_encoder = one_hot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

🎯 What OneHotEncoder does

Suppose your column has:

Geography
France
Spain
Germany

OneHotEncoder converts this into:

France	Spain	Germany
1	    0	    0
0	    1	    0
0	    0	    1

In [8]:
one_hot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [11]:
geo_encoded_df= pd.DataFrame(geo_encoder.toarray(),columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [12]:
## combine one hot encoder with original data
data= pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [13]:
## save the conders and sscaler
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender,file)
with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(one_hot_encoder_geo,file)


🧠 What this does

You are saving your trained encoders to disk using pickle.

This is very important when you build ML models for deployment—
because when new data comes, you must encode it exactly the same way as the training data.

In [16]:
'''
✅ What this code does

You are splitting your dataset into:

X → features (input variables)

y → target label (output variable)
'''

## Divide the dataset in to independent and dependent feature
X = data.drop('Exited', axis=1)
y = data['Exited']

''''
✅ Why do we split the data?

Because:

Training data → used to train the machine learning model

Test data → used to check how well the model performs on unseen data

Without splitting, you cannot measure accuracy properly.

test_size = 0.2
Means:

80% → training data

20% → testing data

If you want 30% test data, you would use test_size=0.3.

random_state=42
This ensures the same random split every time.

👉 universal, funny default seed for randomness

You can use any number:

random_state=1

random_state=99

random_state=2024

It doesn’t matter.

'''


## split the data in training and testing sets
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)


'''
z = x-mean / std

✔ fit() reads training data
  → finds mean of each column
  → finds standard deviation of each column

✔ transform() then scales the training data
'''
## scale these features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]], shape=(8000, 12))

In [17]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

ANN Implementation



This code builds a 3-layer Artificial Neural Network (ANN) using TensorFlow/Keras.
Input → HL1 → HL2 → Output

🔶 Layer 1: Dense(64, activation='relu', input_shape=(X_train.shape[1],))
✔ Dense Layer

Fully connected layer.

✔ Units = 64

This means 64 neurons in this hidden layer.

Each neuron learns different patterns/features.

✔ activation='relu'

ReLU gives non-linearity:

ReLU(x) = max(0, x)


It helps the network learn complex patterns.

✔ input_shape=(X_train.shape[1],)

The network expects as many inputs as there are columns in X_train.

Example: If you have 10 features → input_shape = (10,)

This layer is the bridge from input layer to HL1.


🔶 Layer 2: Dense(32, activation='relu')
✔ 32 neurons

It reduces (compresses) features learned from Layer 1.

✔ activation='relu'

Again helps in learning non-linear relationships.

This is your second hidden layer.

🔶 Output Layer: Dense(1, activation='sigmoid')
✔ Units = 1

You want one output → probability of class 1.

Examples:

churn probability

survived or not

will exit or not

spam or not

✔ activation='sigmoid'
output = 0 to 1


Sigmoid converts raw output into probability.

X_train → rows = samples  
        → columns = features
        If your dataset has:

800 rows (samples)

10 columns (features)

Then:

X_train.shape = (800, 10)


Here:

800 = number of training samples

10 = number of input features

🔥 Why write (X_train.shape[1],) instead of just X_train.shape[1]?

Because:

Keras expects the input shape to be a tuple

Even if there is only one dimension, it must be written like:

(10,)   ✅ correct
10      ❌ incorrect


This defines the shape of one input sample.


In [24]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

## Build Our ANN Model
model = Sequential([
    Dense(64,activation='relu', input_shape=(X_train.shape[1],)), ## HL1 connected with input layer
    Dense(32, activation='relu'),   ## HL2
    Dense(1, activation='sigmoid') ## Output layer
])

In [25]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

🧠 Why create optimizer manually?

You get full control:

change learning rate

add decay

add momentum (for SGD)

customize optimization settings

Example:

opt = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

✅ 2. Creating Loss Function
losses = tensorflow.keras.losses.BinaryCrossentropy()

✔ What does this do?

Binary Crossentropy is the perfect loss for:

sigmoid output

binary classification (0/1)

It measures how far prediction is from true label.

Exactly same as writing:

✅ 3. Compiling Model
model.compile(optimizer=opt, loss=losses, metrics=['accuracy'])

✨ Full Simple Explanation

This code means:

“Train my model using Adam optimizer with learning rate 0.01.
Evaluate error using Binary Crossentropy.
Show accuracy while training.”

In [28]:
## compile modal
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
losses = tensorflow.keras.losses.BinaryCrossentropy()
model.compile(optimizer=opt, loss=losses, metrics=['accuracy'])

This code is used to enable TensorBoard, which is a tool to visualize:

loss

accuracy

learning curves

histograms

model graphs

In [31]:
## set up the tensorboard
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

log_dir = "logs/fit"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


⭐ 1️⃣ What is Early Stopping?

Early stopping automatically stops training when the model stops improving.

Why?

Because if you keep training too long:

the model will start overfitting

validation loss will increase

accuracy will drop on unseen data

Early stopping prevents this.

🔶 monitor='val_loss'

This tells the callback:

“Watch the validation loss during training.”

Validation loss is the best indicator of generalization.

If validation loss stops improving → model is learning noise.

🔶 patience=5

Meaning:

“Wait for 5 epochs even if validation loss is not improving.”

🔶 restore_best_weights=True

This is very important.

When training runs:

weights keep changing every epoch

the best epoch (lowest val_loss) may not be the last epoch

With this option:

After stopping, the model automatically loads the best weights it ever had.

In [35]:
## Set up early stopping
early_stoping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [36]:
## Training modal
history = model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=100,
                    callbacks = [tensorflow_callback, early_stoping_callback])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.8586 - loss: 0.3456 - val_accuracy: 0.8550 - val_loss: 0.3574
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8636 - loss: 0.3380 - val_accuracy: 0.8520 - val_loss: 0.3512
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8593 - loss: 0.3398 - val_accuracy: 0.8655 - val_loss: 0.3477
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8646 - loss: 0.3378 - val_accuracy: 0.8590 - val_loss: 0.3423
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8630 - loss: 0.3324 - val_accuracy: 0.8595 - val_loss: 0.3476
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8635 - loss: 0.3336 - val_accuracy: 0.8600 - val_loss: 0.3507
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8650 - loss: 0.3295 - val_accuracy: 0.8595 - val_loss: 0.3478
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8649 - loss: 0.3307 - val_ac

In [37]:
model.save('model.h5')

In [40]:
## Load tensorboard extension

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [45]:
%tensorboard --logdir logs/fit20251118-222919

Reusing TensorBoard on port 6008 (pid 6468), started 0:00:05 ago. (Use '!kill 6468' to kill it.)